In [1]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Simulate hospital data for the pilot
pilot_hospitals = pd.DataFrame({
    "HospitalID": range(1, 4),
    "Capacity": [50, 70, 60],  # Maximum capacity of each hospital
    "CurrentLoad": [30, 40, 35],  # Current number of patients in each hospital
    "Location": [(43.7, -79.4), (43.8, -79.5), (43.9, -79.6)]  # Latitude, Longitude
})

# Simulate patient data for the pilot
pilot_patients = pd.DataFrame({
    "PatientID": range(1, 21),
    "Urgency": np.random.choice(["High", "Medium", "Low"], 20, p=[0.3, 0.5, 0.2]),  # Random urgency levels
    "Location": [(np.random.uniform(43.6, 43.9), np.random.uniform(-79.6, -79.4)) for _ in range(20)]  # Latitude, Longitude
})

# Function to calculate distances (e.g., using Euclidean for simplicity)
def calculate_distance(loc1, loc2):
    return np.sqrt((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)

# Initial patient-to-hospital matching based on proximity
assignments = []
for patient in pilot_patients.itertuples():
    patient_loc = patient.Location
    
    # Filter hospitals with available capacity
    available_hospitals = pilot_hospitals[pilot_hospitals["CurrentLoad"] < pilot_hospitals["Capacity"]].copy()
    if available_hospitals.empty:
        print(f"Warning: No available hospitals for Patient {patient.PatientID}")
        continue
    
    # Calculate distances to available hospitals
    available_hospitals["Distance"] = available_hospitals["Location"].apply(
        lambda hospital_loc: calculate_distance(patient_loc, hospital_loc)
    )
    
    # Assign the patient to the nearest hospital
    nearest_hospital = available_hospitals.loc[available_hospitals["Distance"].idxmin()]
    assignments.append({"PatientID": patient.PatientID, "AssignedHospital": nearest_hospital["HospitalID"]})
    
    # Update the hospital's current load
    pilot_hospitals.loc[pilot_hospitals["HospitalID"] == nearest_hospital["HospitalID"], "CurrentLoad"] += 1

# Create a DataFrame for assignments
assignment_df = pd.DataFrame(assignments)

# Display the pilot data
print("\n--- Pilot Deployment Data ---")
print("Pilot Hospitals:\n", pilot_hospitals)
print("\nPilot Patients:\n", pilot_patients)
print("\nPatient Assignments:\n", assignment_df)

# Summary of hospital load after assignments
print("\n--- Summary After Assignments ---")
print(pilot_hospitals[["HospitalID", "CurrentLoad", "Capacity"]])



--- Pilot Deployment Data ---
Pilot Hospitals:
    HospitalID  Capacity  CurrentLoad       Location
0           1        50           40  (43.7, -79.4)
1           2        70           50  (43.8, -79.5)
2           3        60           35  (43.9, -79.6)

Pilot Patients:
     PatientID Urgency                                  Location
0           1  Medium   (43.78355586841671, -79.57210122786958)
1           2     Low   (43.68764339456057, -79.52672763134126)
2           3  Medium    (43.73682099526511, -79.4429648077214)
3           4  Medium   (43.65990213464751, -79.49715311231728)
4           5    High     (43.77772437065861, -79.590709917456)
5           6    High   (43.78226345557043, -79.56589517526254)
6           7    High   (43.61951547789558, -79.41022289254934)
7           8     Low  (43.889689609922364, -79.43832053037671)
8           9  Medium   (43.69138413075201, -79.58046557719872)
9          10  Medium  (43.805269907953644, -79.51196950125208)
10         11    High